In [1]:
import pandas as pd
import os
import xgboost as xgb
import numpy as np

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [2]:
# CLR implementation
def clr_(data, eps=1e-6):
    """
    Perform centered log-ratio (clr) normalization on a dataset.

    Parameters:
    data (pandas.DataFrame): A DataFrame with samples as rows and components as columns.

    Returns:
    pandas.DataFrame: A clr-normalized DataFrame.
    """
    if (data < 0).any().any():
        raise ValueError("Data should be strictly positive for clr normalization.")

    # Add small amount to cells with a value of 0
    if (data <= 0).any().any():
        data = data.replace(0, eps)

    # Calculate the geometric mean of each row
    gm = np.exp(data.apply(np.log).mean(axis=1))

    # Perform clr transformation
    clr_data = data.apply(np.log).subtract(np.log(gm), axis=0)

    return clr_data

In [3]:
dir_omics = '../01_data/02_omics/'
dir_env = '../01_data/00_metadata/'

dir_out_predictions = '../03_results/out_predictions'
os.makedirs(dir_out_predictions, exist_ok=True)

metadata_file = os.path.join(dir_env, 'metadata_chile_cont.tsv')
md = pd.read_csv(metadata_file, sep='\t', index_col=0)
md_all = pd.read_csv(os.path.join(dir_env,'metadata_chile.tsv'), sep='\t', index_col=0)

omics_files = [f for f in os.listdir(dir_omics) if f.startswith('Matrix_TF_') and f.endswith('_all.tsv')]

## Binning

In [4]:
# Creating the new column 'Latitude Bin' in md_all
bins = [-float('inf'), -40, -30, float('inf')]
labels = ['South', 'Center', 'North']
md_all['Latitude Bin'] = pd.cut(md_all['lat_cast'], bins=bins, labels=labels)

binning_params = {
    'Temperature [ºC]': {
        'bins': [-float('inf'), 10, 15, float('inf')],
        'labels': ['Low', 'Mid', 'High']
    },
    'Salinity [PSU]': {
        'bins': [-float('inf'), 34.25, float('inf')],
        'labels': ['Low', 'High']
    },
    'Oxygen [ml/l]': {
        'bins': [-float('inf'), 2.5, 5, float('inf')],
        'labels': ['Low', 'Mid', 'High']
    },
    'Oxygen [%]': {
        'bins': [-float('inf'), 30, 80, float('inf')],
        'labels': ['Low', 'Mid', 'High']
    },
    'Fluorescence [mg/m3]': {
        'bins': [-float('inf'), 3, float('inf')],
        'labels': ['Low', 'High']
    },
    'Orthophosphate [uM]': {
        'bins': [-float('inf'), 1.4, 2.5, float('inf')],
        'labels': ['Low', 'Mid', 'High']
    },
    'Silicic-acid [uM]': {
        'bins': [-float('inf'), 7, 22, float('inf')],
        'labels': ['Low', 'Mid', 'High']
    },
    'Nitrite [uM]': {
        'bins': [-float('inf'), 0.1, float('inf')],
        'labels': ['Low', 'High']
    },
    'Nitrates [uM]': {
        'bins': [-float('inf'), 11, 25, float('inf')],
        'labels': ['Low', 'Mid', 'High']
    },
    'Nitrate [uM]': {
        'bins': [-float('inf'), 11, 25, float('inf')],
        'labels': ['Low', 'Mid', 'High']
    },
    'NP ratio': {
        'bins': [-float('inf'), 7.5, 12.5, float('inf')],
        'labels': ['Low', 'Mid', 'High']
    }
}

# Apply binning to each relevant column in md
for column, params in binning_params.items():
    md_all[column + ' Bins'] = pd.cut(md[column], bins=params['bins'], labels=params['labels'])

# Displaying the updated dataframe to ensure the binnings are applied correctly
md_all.columns

Index(['SAMEA ID', 'Leg', 'Station', 'Station ID', 'Depth ID', 'lat_cast',
       'lon_cast', 'datetime', 'Depth [m]', 'Temperature [ºC]',
       'Salinity [PSU]', 'Density [kg/m3]', 'Oxygen [ml/l]', 'Oxygen [%]',
       'Fluorescence [mg/m3]', 'Orthophosphate [uM]', 'Silicic-acid [uM]',
       'Nitrite [uM]', 'Nitrates [uM]', 'Nitrate [uM]', 'NP ratio', 'year',
       'month', 'day', 'hour', 'minute', 'second', 'instrument',
       'original file', 'Depth level', 'Oxygen level', 'Biogeographical units',
       'Freshwater inputs', 'Oxy_depth', 'Distance from coast (km)',
       'Latitude Bin', 'Temperature [ºC] Bins', 'Salinity [PSU] Bins',
       'Oxygen [ml/l] Bins', 'Oxygen [%] Bins', 'Fluorescence [mg/m3] Bins',
       'Orthophosphate [uM] Bins', 'Silicic-acid [uM] Bins',
       'Nitrite [uM] Bins', 'Nitrates [uM] Bins', 'Nitrate [uM] Bins',
       'NP ratio Bins'],
      dtype='object')

# Classification

In [5]:
target_columns = [
    'Temperature [ºC] Bins', 'Salinity [PSU] Bins', 'Oxygen [ml/l] Bins',
    'Oxygen [%] Bins', 'Fluorescence [mg/m3] Bins', 'Orthophosphate [uM] Bins',
    'Silicic-acid [uM] Bins', 'Nitrite [uM] Bins', 'Nitrates [uM] Bins',
    'Nitrate [uM] Bins', 'NP ratio Bins', 'Depth level', 'Oxygen level', 'Oxy_depth', 'Latitude Bin'
]

for omics_file in omics_files:
    omics_path = os.path.join(dir_omics, omics_file)
    omics_data = pd.read_csv(omics_path, sep='\t', index_col=0)
    
    # Align metadata and omics data based on common samples
    common_samples = md_all.index.intersection(omics_data.index)
    aligned_md = md_all.loc[common_samples]
    aligned_omics = omics_data.loc[common_samples]
    
    # Apply clr transformation
    clr_omics = clr_(aligned_omics)
    
    # Store results in a DataFrame
    results = []

    for target in target_columns:
        X = clr_omics
        y = aligned_md[target].dropna()
        X = X.loc[y.index]
        
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y)
        
        rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
        accuracies = []
        f1_scores = []
        
        for train_index, test_index in rkf.split(X, y_encoded):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y_encoded[train_index], y_encoded[test_index]
            
            model = xgb.XGBClassifier(
                objective='multi:softmax',
                num_class=len(label_encoder.classes_),
                eval_metric='mlogloss'
            )
            model.fit(X_train, y_train)
            
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            
            accuracies.append(accuracy)
            f1_scores.append(f1)
        
        mean_accuracy = np.mean(accuracies)
        std_accuracy = np.std(accuracies)
        mean_f1 = np.mean(f1_scores)
        std_f1 = np.std(f1_scores)
        
        # Print the results
        print(f"Target: {target}")
        print(f"Mean Accuracy: {mean_accuracy:.2f}")
        print(f"Standard Deviation of Accuracy: {std_accuracy:.2f}")
        print(f"Mean F1 Score: {mean_f1:.2f}")
        print(f"Standard Deviation of F1 Score: {std_f1:.2f}")
        print("----------")
        
        # Collect results in a list of dictionaries
        results.append({
            'Target': target,
            'Mean Accuracy': mean_accuracy,
            'Std Accuracy': std_accuracy,
            'Mean F1 Score': mean_f1,
            'Std F1 Score': std_f1
        })
    
    # Convert results to DataFrame and save as TSV
    results_df = pd.DataFrame(results)
    output_file = os.path.join(dir_out_predictions, f'{os.path.splitext(omics_file)[0]}_performance.tsv')
    results_df.to_csv(output_file, sep='\t', index=False)

    print(f"Results saved for {omics_file} in {output_file}")

Target: Temperature [ºC] Bins
Mean Accuracy: 0.77
Standard Deviation of Accuracy: 0.07
Mean F1 Score: 0.75
Standard Deviation of F1 Score: 0.08
----------
Target: Salinity [PSU] Bins
Mean Accuracy: 0.82
Standard Deviation of Accuracy: 0.06
Mean F1 Score: 0.82
Standard Deviation of F1 Score: 0.06
----------
Target: Oxygen [ml/l] Bins
Mean Accuracy: 0.70
Standard Deviation of Accuracy: 0.09
Mean F1 Score: 0.70
Standard Deviation of F1 Score: 0.09
----------
Target: Oxygen [%] Bins
Mean Accuracy: 0.71
Standard Deviation of Accuracy: 0.08
Mean F1 Score: 0.71
Standard Deviation of F1 Score: 0.08
----------
Target: Fluorescence [mg/m3] Bins
Mean Accuracy: 0.80
Standard Deviation of Accuracy: 0.09
Mean F1 Score: 0.80
Standard Deviation of F1 Score: 0.09
----------
Target: Orthophosphate [uM] Bins
Mean Accuracy: 0.72
Standard Deviation of Accuracy: 0.07
Mean F1 Score: 0.71
Standard Deviation of F1 Score: 0.07
----------
Target: Silicic-acid [uM] Bins
Mean Accuracy: 0.65
Standard Deviation of A